In [1]:
import json
from osgeo import ogr
from shapely.geometry import mapping, shape
import xml.etree.ElementTree as ET

ogr.UseExceptions()

raw = json.load(open("raw/properties.geojson"))["features"]

In [2]:
# merge properties that have the same address

addresses = {}
for p in raw:
    # ignore properties that do not have an address
    if p["properties"]["houseno"] == 0:
        continue
    # some nature reserves have house numbers but should still be ignored
    if p["properties"]["propertytype"] == "RESERVE":
        continue

    
    
    a = p["properties"]["houseaddress"]
    if a in addresses:
        addresses[a]["geometry"] = mapping(
            shape(addresses[a]["geometry"]).union(shape(p["geometry"]))
        )
        addresses[a]["properties"]["propertyno"] = (
            str(addresses[a]["properties"]["propertyno"])
            + ";"
            + str(p["properties"]["propertyno"])
        )
    else:
        addresses[a] = p

In [3]:
# convert and export

root = ET.Element("osm", version="0.6", generator="JOSM")
xml_id = -1

for a, p in addresses.items():
    ps = p["properties"]
    geom = ogr.CreateGeometryFromJson(json.dumps(p["geometry"]))

    tags = {
        "addr:housenumber": str(ps["houseno"]),
        "addr:street": ps["streetname"],
        "source:addr": "Townsville City Council",
    }
    
    if ps["housenoto"] == 0:
        tags["addr:housenumber"] += ps["housenosuffix"]
    else:
        tags["addr:housenumber"] += "-" + str(ps["housenoto"])
        
    if ps["unitno"] != 0:
        tags["addr:unit"] = str(ps["unitno"]) + ps["unitnosuffix"]
        if ps["unitnoto"] != 0:
            tags["addr:unit"] += "-" + str(ps["unitnoto"]) + ps["unitnotosuffix"]

    # generate XML for JOSM
    point = geom.PointOnSurface().GetPoint()
    node = ET.SubElement(
        root, "node", id=str(xml_id), lat=str(point[1]), lon=str(point[0])
    )
    xml_id -= 1
    for k, v in tags.items():
        ET.SubElement(node, "tag", k=k, v=str(v))


ET.ElementTree(root).write("osm/addresses.osm")